In [1]:
from typing import TypedDict, List, Optional
from langgraph.graph import StateGraph
from openai import OpenAI
from langchain_community.chat_models import ChatOpenAI

# Définition de l'état
class CVGenerationState(TypedDict):
    profile: dict             # Données du profil
    job: dict                # Données sur le job
    current_step: str
    error: Optional[str]

# Exemple de structure pour les données
# profile = {
#     "head_raw": dict,           # Informations de l'en-tête
#     "skills_raw": dict,         # Compétences
#     "experiences": List[dict],  # Liste d'expériences
#     "education": List[dict],    # Liste de formations
#     "hobbies_raw": dict         # Loisirs
# }
# job = {
#     "job_posting": str,     # Fiche de poste
#     "job_summary": Optional[str] # Résumé de la fiche de poste
# }

# Structure pour une expérience dans profile["experiences"]
# experience = {
#     "titre_raw": str,
#     "titre_refined": str,
#     "dates_raw": str,
#     "dates_refined": str,
#     "entreprise_raw": str,
#     "entreprise_refined": str,
#     "lieu_raw": str,
#     "lieu_refined": str,
#     "description": str,         # Description brute
#     "sumup": str,              # Résumé
#     "bullets_points": List[str], # Points clés
#     "poid": float              # Poids en %
# }

# Structure pour une formation dans profile["education"]
# education = {
#     "titre_raw": str,
#     "titre_refined": str,
#     "dates_raw": str,
#     "dates_refined": str,
#     "etablissement_raw": str,
#     "etablissement_refined": str,
#     "lieu_raw": str,
#     "lieu_refined": str,
#     "description": str,         # Description brute
#     "sumup": str,              # Résumé
#     "poid": float              # Poids en %
# }


In [2]:
# Définition du modèle de langage
llm = ChatOpenAI(
    model="gpt-4o-mini",
    temperature=0,
)

# Test du modèle
test_response = llm.invoke("Dis bonjour en français")
print("Test de réponse du modèle:")
print(test_response.content)


/var/folders/kz/q2b2x49j16x2jfrqxg8_grdc0000gn/T/ipykernel_12360/2207125639.py:2: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm = ChatOpenAI(


Test de réponse du modèle:
Bonjour !


In [3]:
import json

# Chargement des données de test depuis le fichier JSON
with open('data_test.json', 'r', encoding='utf-8') as json_file:
    test_data = json.load(json_file)

# Chargement de la fiche de poste depuis le fichier texte
with open('fiche_poste.txt', 'r', encoding='utf-8') as text_file:
    job_posting = text_file.read()

# Initialisation de l'état avec les données de test et la fiche de poste
initial_state = {
    "profile": {
        "head_raw": test_data["cv_data"]["head"],
        "skills_raw": test_data["cv_data"]["skills"],
        "experiences": [
            {
                "titre_raw": exp["intitule"],
                "titre_refined": None,
                "dates_raw": exp["dates"],
                "dates_refined": None,
                "entreprise_raw": exp["etablissement"],
                "entreprise_refined": None,
                "lieu_raw": exp["lieu"],
                "lieu_refined": None,
                "description": exp["description"],
                "sumup": None,
                "bullets_points": [],
                "poid": None
            }
            for exp in test_data["cv_data"]["experiences"]["experiences"]
        ],
        "education": [
            {
                "titre_raw": edu["intitule"],
                "titre_refined": None,
                "dates_raw": edu["dates"],
                "dates_refined": None,
                "etablissement_raw": edu["etablissement"],
                "etablissement_refined": None,
                "lieu_raw": edu["lieu"],
                "lieu_refined": None,
                "description": edu["description"],
                "sumup": None,
                "poid": None
            }
            for edu in test_data["cv_data"]["education"]["educations"]
        ],
        "hobbies_raw": test_data["cv_data"]["hobbies"]
    },
    "job": {
        "job_posting": job_posting,
        "job_summary": None
    }
}

In [4]:
# Nodes (fonctions de transformation)
def summarize_job(state: CVGenerationState) -> CVGenerationState:
    response = llm.invoke(
        "Faites un résumé de cette fiche de poste en 100 mots maximum, en incluant les informations principales pour rédiger un CV.\n\n" + 
        state["job"]["job_posting"]
    )
    state["job"]["job_summary"] = response.content.strip()
    return state


In [5]:
initial_state = summarize_job(initial_state)

# on affiche le résumé de la fiche de poste
print(initial_state["job"]["job_summary"])


**Résumé de la fiche de poste - Machine Learning Engineer**

Poste : Machine Learning Engineer en CDI, Paris (hybride). Expérience requise : 3+ ans. Salaire : 55k - 75k €. Startup innovante en IA et Data, développant des solutions de Machine Learning et Deep Learning. Missions : concevoir et déployer des modèles ML, optimiser pipelines de données, développer des API, et collaborer avec des équipes pluridisciplinaires. Compétences requises : Python (TensorFlow, PyTorch), SQL, AWS, MLOps (MLflow, Kubeflow), et bonnes pratiques en ingénierie ML. Diplôme en Informatique ou équivalent souhaité. Environnement dynamique et projets innovants.


In [6]:
def summarize_profile(state: CVGenerationState) -> CVGenerationState:
    # Résumer chaque expérience
    for experience in state["profile"]["experiences"]:
        response = llm.invoke(
            "Faites un résumé de cette expérience en 50 mots maximum.\n\n" + 
            experience["description"]
        )
        experience["sumup"] = response.content.strip()
    
    # Résumer chaque formation
    for education in state["profile"]["education"]:
        response = llm.invoke(
            "Faites un résumé de cette formation en 50 mots maximum.\n\n" + 
            education["description"]
        )
        education["sumup"] = response.content.strip()
    
    return state

#tester la fonction sur initial_state
initial_state = summarize_profile(initial_state)

#afficher les résumés
for experience in initial_state["profile"]["experiences"]:
    print("Résumé de l'expérience :", experience.get("sumup", "Pas de résumé disponible."))


Résumé de l'expérience : Mission chez Renault (FLMDH) visant à moderniser et structurer les équipes de data engineering. Pilotage de projets essentiels pour soutenir la transition vers le leasing et les véhicules électriques.
Résumé de l'expérience : Développement d'une application mobile innovante pour l'analyse automatique de tickets de caisse, incluant la définition de l'architecture technique et l'implémentation de solutions d'intelligence artificielle. Cette expérience met en avant des compétences en gestion de projet et en technologies avancées.
Résumé de l'expérience : Gestion de projets stratégiques visant à moderniser l'entreprise par la digitalisation et l'optimisation des processus internes, notamment dans le domaine de la fabrication, pour améliorer l'efficacité et la performance globale.
Résumé de l'expérience : Participation à des missions stratégiques visant à optimiser la supply chain en utilisant des approches quantitatives pour résoudre des problématiques complexes.
R

In [7]:
#afficher les résumés des éducations
for education in initial_state["profile"]["education"]:
    print("Résumé de l'éducation :", education.get("sumup", "Pas de résumé disponible."))


Résumé de l'éducation : Cette formation de haut niveau allie des compétences scientifiques et techniques, axées sur la supply chain et le machine learning. Elle prépare les participants à relever des défis complexes dans ces domaines, en leur fournissant des connaissances approfondies et des outils pratiques pour optimiser les processus et l'analyse de données.
Résumé de l'éducation : Cette formation permet d'approfondir les connaissances en mathématiques appliquées, finance computationnelle, machine learning et gestion de la chaîne d'approvisionnement, offrant ainsi des compétences essentielles pour analyser des données complexes et optimiser des processus dans divers secteurs.
Résumé de l'éducation : Formation académique rigoureuse visant à préparer les étudiants aux concours des grandes écoles d'ingénieurs, en développant des compétences techniques et scientifiques essentielles. Elle combine cours théoriques, travaux pratiques et entraînements aux épreuves, afin d'assurer une solide

In [8]:
def select_experiences(state: CVGenerationState) -> CVGenerationState:
    # Préparer le contexte pour l'IA
    job_context = state["job"]["job_summary"]
    experiences_context = "\n".join([
        f"- {exp['titre_raw']} à {exp['lieu_raw']} ({exp['dates_raw']}) chez {exp['entreprise_raw']}: {exp['sumup']}" 
        for exp in state["profile"]["experiences"]
    ])
    
    # Premier LLM pour sélectionner et pondérer les expériences
    prompt = f"""
    En tant qu'expert RH, analysez les expériences du candidat par rapport au poste.
    Sélectionnez les expériences les plus pertinentes et attribuez-leur un pourcentage d'importance (la somme doit faire 100%).
    Répondez sous forme de texte simple en listant les expériences retenues avec leur pourcentage.
    
    Poste à pourvoir:
    {job_context}
    
    Expériences du candidat:
    {experiences_context}
    """
    
    selection_response = llm.invoke(prompt)
    selection_text = selection_response.content
    
    # Pour chaque expérience, demander au second LLM d'extraire le poids
    for exp in state["profile"]["experiences"]:
        exp_prompt = f"""
        Voici la sélection d'expériences pertinentes avec leurs pourcentages:
        {selection_text}
        
        Pour l'expérience suivante, retournez uniquement le pourcentage attribué (juste le nombre), ou "null" si elle n'est pas mentionnée:
        {exp['titre_raw']} chez {exp['entreprise_raw']}
        """
        
        weight_response = llm.invoke(exp_prompt)
        try:
            weight = float(weight_response.content.strip())
            exp["poid"] = weight
        except ValueError:
            exp["poid"] = None
    
    return state

# Tester la fonction
initial_state = select_experiences(initial_state)

# Afficher les expériences sélectionnées
print("\nExpériences sélectionnées et leur poids:")
for exp in initial_state["profile"]["experiences"]:
    weight = exp.get("poid")
    if weight is not None:
        print(f"- {exp['titre_raw']} chez {exp['entreprise_raw']}: {weight}%")
    else:
        print(f"- {exp['titre_raw']} chez {exp['entreprise_raw']}: Non retenue")



Expériences sélectionnées et leur poids:
- Senior Data Scientist chez EY: 30.0%
- Co-fondateur et CTO chez Kadi: 25.0%
- Ingénieur en Transformation Digitale chez Blispac: 15.0%
- Consultant en Stratégie et Supply Chain chez Diagma: 10.0%
- Analyste de Marché chez Total: 10.0%
- Coordinateur de Projets chez L’Œuvre d’Orient: 5.0%
- Consultant junior chez Cdiscount: 5.0%


In [12]:
print("\nÉtat actuel:")
print(json.dumps(initial_state, indent=2, ensure_ascii=False))



État actuel:
{
  "profile": {
    "head_raw": {
      "general_title": "Data Engineer / Data Scientist | Expert Cloud et Data. Alexis de Monts est un ingénieur des Ponts et Chaussées, spécialisé en Data Science, qui aide les entreprises à transformer leurs données en valeur. Avec une expertise approfondie dans les domaines du Cloud (GCP, AWS), DevOps (CI/CD, Docker) et MLOps, il intègre des solutions d'intelligence artificielle et de machine learning dans des systèmes informatiques complexes. Son parcours professionnel est marqué par des réalisations significatives, telles que la création d'un Data Hub centralisé pour Renault et le développement d'une application mobile innovante intégrant une IA d'analyse.",
      "email": "alexis.demonts.s@gmail.com",
      "phone": "07 81 37 86 80",
      "name": "Alexis de Monts"
    },
    "skills_raw": {
      "description": "Alexis possède un large éventail de compétences techniques et managériales. En tant que Senior Data Scientist chez EY, il

In [10]:
def select_education(state):
    """
    Sélectionne et pondère les formations pertinentes pour le poste.
    """
    # Récupérer le contexte
    job_context = state["job"]["job_summary"]
    education_context = "\n".join([f"{edu['titre_raw']} - {edu['etablissement_raw']} ({edu['dates_raw']}) {edu['lieu_raw']} {edu['sumup']}" 
                                 for edu in state["profile"]["education"]])
    
    # Créer le prompt
    prompt = f"""
    En tant qu'expert RH, analysez les formations du candidat et sélectionnez celles qui sont les plus pertinentes 
    pour le poste à pourvoir. Attribuez un pourcentage à chaque formation selon sa pertinence.

    Formations du candidat:
    {education_context}

    Poste à pourvoir:
    {job_context}
    """
    
    selection_response = llm.invoke(prompt)
    selection_text = selection_response.content
    
    # Pour chaque formation, demander au LLM d'extraire le poids
    for edu in state["profile"]["education"]:
        edu_prompt = f"""
        Voici la sélection de formations pertinentes avec leurs pourcentages:
        {selection_text}
        
        Pour la formation suivante, retournez uniquement le pourcentage attribué (juste le nombre), ou "null" si elle n'est pas mentionnée:
        {edu['titre_raw']} à {edu['etablissement_raw']}
        """
        
        weight_response = llm.invoke(edu_prompt)
        try:
            weight = float(weight_response.content.strip())
            edu["poid"] = weight
        except ValueError:
            edu["poid"] = None
    
    return state

# Tester la fonction
initial_state = select_education(initial_state)

# Afficher les formations sélectionnées
print("\nFormations sélectionnées et leur poids:")
for edu in initial_state["profile"]["education"]:
    weight = edu.get("poid")
    if weight is not None:
        print(f"- {edu['titre_raw']} à {edu['etablissement_raw']}: {weight}%")
    else:
        print(f"- {edu['titre_raw']} à {edu['etablissement_raw']}: Non retenue")



Formations sélectionnées et leur poids:
- Diplôme d’ingénieur à École des Ponts ParisTech: 40.0%
- Master 2 en Mathematical Engineering à Politecnico di Milano: 50.0%
- Classe préparatoire scientifique à Collège Stanislas Paris: 20.0%


In [11]:
def generate_experience_bullets(state):
    """
    Génère des bullet points pour chaque expérience en fonction de leur poids
    """
    if "cv" not in state or "experiences_refined" not in state["cv"]:
        return state
    
    for exp in state["cv"]["experiences_refined"]:
        # Calculer le nombre de bullet points en fonction du weight
        nb_bullets = max(2, min(6, round(exp["weight"] / 20)))
        
        prompt = f"""
        En tant qu'expert RH, générez {nb_bullets} bullet points percutants qui mettent en valeur cette expérience 
        professionnelle par rapport au poste visé.
        
        Chaque bullet point doit:
        - Commencer par un verbe d'action fort
        - Être concis et impactant
        - Mettre en avant les réalisations concrètes
        - Être adapté au poste visé
        
        Répondez au format JSON suivant:
        {{
            "bullet_points": [
                "Premier bullet point",
                "Deuxième bullet point",
                etc.
            ]
        }}
        
        Poste visé (résumé):
        {state["job"]["job_summary"]}
        
        Expérience à décrire:
        {exp["description"]}
        """
        
        response = llm.invoke(input=prompt, response_format={"type": "json_object"})
        bullets = json.loads(response.content)
        
        # Ajouter les bullet points à l'expérience
        exp["bullet_points"] = bullets["bullet_points"]
    
    return state

# Tester la fonction
initial_state = generate_experience_bullets(initial_state)

# Afficher les bullet points générés
print("\nBullet points générés pour chaque expérience:")
for exp in initial_state["cv"]["experiences_refined"]:
    print(f"\n{exp['poste']} chez {exp['entreprise']}:")
    for bullet in exp["bullet_points"]:
        print(f"• {bullet}")



Bullet points générés pour chaque expérience:


KeyError: 'cv'

In [ ]:
def create_cv_generation_graph():
    # Création du graph
    graph = StateGraph()
    
    # Ajout des nodes
    graph.add_node("summarize_job", summarize_job)
    graph.add_node("summarize_profile", summarize_profile)
    
    # Configuration du flux
    graph.set_entry_point("summarize_job")
    graph.add_edge("summarize_job", "summarize_profile")
    
    # Compiler le graph
    chain = graph.compile()
    return chain
